In [ ]:
import os
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [ ]:
from ignite.engine import Events
from ignite.metrics import Loss
from ignite.contrib.handlers.param_scheduler import LRScheduler

from segmentation.datasets import MobiActV2
from segmentation.models import SensorFCN
from segmentation.engine import create_trainer, create_evaluator
from segmentation.metrics import (
    SamplewiseAccuracy,
    MeanAccuracy,
    MeanIoU,
    FrequencyWeightedIoU,
)
from segmentation.logger import Logger
from segmentation import utils

In [ ]:
# Papermill parameter cell
# Model params
sensors = "ago"
input_kernel_size = 5
n_filters = 16
smoothing_kernel_size = 0

# Run params
random_seed = 1234
experiment = "input_kernel_size"
gpu = "cuda:3"

In [ ]:
sensor_channels = []
if "a" in sensors:
    sensor_channels.extend(["acc_x", "acc_y", "acc_z"])
if "g" in sensors:
    sensor_channels.extend(["gyro_x", "gyro_y", "gyro_z"])
if "o" in sensors:
    sensor_channels.extend(["azimuth", "pitch", "roll"])

In [ ]:
# Train/validation/test split
np.random.seed(random_seed)
users = np.arange(1, 68)
np.random.shuffle(users)

test_users = users[0:7]
validation_users = users[7:14]
train_users = users[14:]

# Load datasets
train_set = MobiActV2("data/MobiActV2/frames", sensor_channels, train_users)
validation_set = MobiActV2("data/MobiActV2/frames", sensor_channels, validation_users)
test_set = MobiActV2("data/MobiActV2/frames", sensor_channels, test_users)
n_classes = len(train_set.label_codes)

# Define data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

In [ ]:
# Define device
device = torch.device(gpu if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Initialize network
model = SensorFCN(
    n_input_channels=len(sensor_channels),
    n_classes=int(n_classes),
    input_kernel_size=int(input_kernel_size),
    n_filters=int(n_filters),
    smoothing_kernel_size=int(smoothing_kernel_size),
)

In [ ]:
# Load class weights
class_weights = np.load("data/MobiActV2/class_weights.npy")
class_weights = torch.tensor(class_weights).to(device, dtype=torch.float)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Create trainer and evaluator engines
trainer = create_trainer(model, optimizer, loss_fn, device)
evaluator = create_evaluator(
    model,
    device,
    metrics={
        "loss": Loss(loss_fn),
        "samplewise_accuracy": SamplewiseAccuracy(),
        "mean_accuracy": MeanAccuracy(),
        "mean_iou": MeanIoU(),
        "frequency_weighted_iou": FrequencyWeightedIoU(),
    },
)

# Attach LR scheduler
step_scheduler = StepLR(optimizer, step_size=5, gamma=0.9)
scheduler = LRScheduler(step_scheduler)
trainer.add_event_handler(Events.EPOCH_COMPLETED, scheduler)

# Attach handler for training logging
logger = Logger(evaluator, train_loader, validation_loader)
trainer.add_event_handler(Events.EPOCH_COMPLETED, logger)

# Run trainer engine
trainer.run(train_loader, max_epochs=50)

## Training metrics

In [ ]:
utils.plot_metrics(logger.metrics, "training")

## Validation metrics

In [ ]:
utils.plot_metrics(logger.metrics, "validation")

## Training confusion matrix

In [ ]:
y_true_tr, y_pred_tr = utils.predict_with_model(model, train_set, device)
utils.plot_confusion_matrix(
    y_true_tr, y_pred_tr, list(train_set.label_codes.keys()), normalize=True
)

## Validation confusion matrix

In [ ]:
y_true_val, y_pred_val = utils.predict_with_model(model, validation_set, device)
utils.plot_confusion_matrix(
    y_true_val, y_pred_val, list(validation_set.label_codes.keys()), normalize=True
)

## Save data

In [ ]:
if experiment == "input_kernel_size":
    name = str(input_kernel_size)
if experiment == "n_filters":
    name = str(n_filters)
if experiment == "sensors":
    name = sensors
if experiment == "smoothing_kernel_size":
    name = f"{str(smoothing_kernel_size)}"

output_dir = f"output_asd/{random_seed}/{experiment}/{name}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Save trained model
torch.save(model.state_dict(), os.path.join(output_dir, "model.pt"))

# Save model log/history
f = open(os.path.join(output_dir, "hist.p"), "wb")
pickle.dump(logger.metrics, f)
f.close()

# Save number of parameters
f = open(os.path.join(output_dir, "n_params.p"), "wb")
pickle.dump(sum(p.numel() for p in model.parameters()), f)
f.close()

# Save training and validation ground truth and predictions
np.save(os.path.join(output_dir, "y_true_tr.npy"), y_true_tr)
np.save(os.path.join(output_dir, "y_pred_tr.npy"), y_pred_tr)
np.save(os.path.join(output_dir, "y_true_val.npy"), y_true_val)
np.save(os.path.join(output_dir, "y_pred_val.npy"), y_pred_val)